## Envio de dados para o Google BigQuery via Python

O banco de dados utilizado foi o de alugueis de apartamentos pelo Airbnb presente na plataforma da Kaggle.

O intuito aqui é mostrar como é fácil enviar arquivos via python para a plataforma BigQuery

In [8]:
#Primeiro vamos importar as bibliotecas e o banco de dados
import pandas as pd
import pandas_gbq
from google.oauth2 import service_account

df = pd.read_csv("Projeto Airbnb NY/archive/AB_NYC_2019.csv")

O pandas vamos utilizar para carregar a base de dados, o pandas_gbq vamos usar para enviar os dados para o BigQuery e o google.oauth2 vamos usar para validar a autenticação que o google exige para que possamos fazer a conexão o nosso programa local.

In [9]:
#Verificando a base de dados
df.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Para que possamos realizar a conexão com o Google Cloud precisamos das credenciais que o mesmo disponibiliza.
Então dentro do Google Cloud devemos baixar o arquivo JSON que vai servir como chave para que o nosso banco de dados possa entrar na plataforma.

Nesse exemplo salvei o arquivo como GCP.json e coloquei na mesma pasta do nosso script

In [15]:
#Especificando a nossa chave json para conexão e criando a credencial
key_path = "GCP.json"

credentials = service_account.Credentials.from_service_account_file(key_path, 
                             scopes = 'http://www.googleapis.com/auth/coud-plataform')

In [16]:
#Enviando o arquivo para a pasta "unificado" dentro do meu projeto no BigQuery
pandas_gbq.to_gbq(df, 'unificados.airbnbny', 
                  project_id = "projeto1-350801",
                  if_exists='replace')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=pdG56DsZspUUU8XAtuuLGPO8OfifpK&access_type=offline


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Ao rodar o código acima será solicitado o login na sua conta Google.
Após a autenticação na conta do google o arquivo será enviado para o Bigquery

Agora vamos fazer a leitura desses dados no Bigquery via python

In [32]:
#Para a leitura desses dados só precisamos passar o local do arquivo dentro do comando abaixo
#Como já realizamos as autenticações não precisamos criar as credenciais novamente
dados_gbq = pandas_gbq.read_gbq("projeto1-350801.unificados.airbnbny")

Downloading: 100%|███████████████████████████████████████████████████████████| 48895/48895 [00:20<00:00, 2349.92rows/s]


In [33]:
dados_gbq.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,55668,"NOHO/EAST VILLAGE, PRIVATE 1/2 BATH",88209,Jason,Manhattan,NoHo,40.72773,-73.99134,Private room,130,2,115,2018-12-05,1.17,1,75
1,803778,Luxury Loft Noho New York City,4230317,Jenny,Manhattan,NoHo,40.72591,-73.99452,Entire home/apt,465,30,47,2019-05-31,0.67,1,277
2,1818411,HUGE 2bdrm LOFT in NOHO/East Vill!,9522475,Liam,Manhattan,NoHo,40.72569,-73.99227,Entire home/apt,455,30,93,2019-01-03,1.36,2,332
3,2201154,Prime E. Village at St. Marks Place,5081260,Eden,Manhattan,NoHo,40.72780,-73.99205,Private room,208,2,38,2019-06-18,0.60,1,313
4,6747685,Beautiful 1205 ft classic NoHo Loft,29769754,Tom,Manhattan,NoHo,40.72590,-73.99390,Entire home/apt,499,3,35,2019-05-27,0.72,1,223
